# Final Bootes master catalogue

This notebook presents the combination of the HELP standard catalogue with the aperture matched photometry from Brown which gives superior photometric redhsifts.

In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))
import datetime
print("This notebook was executed on: \n{}".format(datetime.datetime.now()))

This notebook was run with herschelhelp_internal version: 
017bb1e (Mon Jun 18 14:58:59 2018 +0100)
This notebook was executed on: 
2019-02-01 11:42:10.977732


In [2]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

import os
import time

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table, join
import numpy as np
from pymoc import MOC

from herschelhelp_internal.masterlist import merge_catalogues, nb_merge_dist_plot, specz_merge
from herschelhelp_internal.utils import coords_to_hpidx, ebv, gen_help_id, inMoc

/opt/pyenv/versions/3.7.2/lib/python3.7/site-packages/matplotlib/__init__.py:855: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
/opt/pyenv/versions/3.7.2/lib/python3.7/site-packages/matplotlib/__init__.py:846: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 2.2 and will be removed in 3.1.
  "2.2", name=key, obj_type="rcparam", addendum=addendum)
/opt/pyenv/versions/3.7.2/lib/python3.7/site-packages/seaborn/apionly.py:9: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


In [3]:
TMP_DIR = os.environ.get('TMP_DIR', "./data_tmp")
OUT_DIR = os.environ.get('OUT_DIR', "./data")
SUFFIX = os.environ.get('SUFFIX', time.strftime("_%Y%m%d"))

try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

## I - Reading the prepared pristine catalogues

In [4]:
brown_catalogue_name ="master_catalogue_bootes_brown_20180517.fits"
help_catalogue_name ="master_catalogue_bootes_help_20180517.fits"

brown_cross_id_name = "master_list_cross_ident_bootes_brown_20180517.fits"
help_cross_id_name = "master_list_cross_ident_bootes_help_20180517.fits"

In [5]:
brown_catalogue = Table.read("{}/{}".format(OUT_DIR, brown_catalogue_name))
help_catalogue = Table.read("{}/{}".format(OUT_DIR, help_catalogue_name))

In [6]:
brown_cross_id = Table.read("{}/{}".format(OUT_DIR,brown_cross_id_name))['full_help_id', 'help_id']
brown_cross_id['help_id'].name = 'brown_help_id'
brown_cross_id['full_help_id'].name = 'help_help_id'

In [7]:
in_brown = set(brown_catalogue.colnames)
in_help = set(help_catalogue.colnames)
in_both = in_help.intersection(in_brown)
only_in_brown = in_brown - in_help
only_in_help = in_help - in_brown

for col in brown_catalogue.colnames:
    brown_catalogue[col].name = "brown_{}".format(col)
    
for col in help_catalogue.colnames:
    help_catalogue[col].name = "help_{}".format(col)

## 2 - Join the two catalogues taking brown if both are available

In [8]:
master_catalogue = join(brown_cross_id, brown_catalogue, join_type='left')
master_catalogue = join(master_catalogue, help_catalogue, join_type='left')
             



In [9]:
in_both.remove('help_id')
for col in in_both:
    has_brown = (master_catalogue['brown_help_id'] != '-1')
                
    try:
        has_brown = has_brown  & ~np.isnan(master_catalogue["brown_{}".format(col)])
    except TypeError:
        print("brown_{} is not a column of floats".format(col))
    master_catalogue["help_{}".format(col)][has_brown] = master_catalogue["brown_{}".format(col)][has_brown]
    master_catalogue["help_{}".format(col)].name = col
    master_catalogue.remove_column("brown_{}".format(col))
    
#Deal with ids seperately
has_brown = (master_catalogue['brown_help_id'] != '-1')
master_catalogue["help_help_id"][has_brown] = master_catalogue["brown_help_id"][has_brown]
master_catalogue["help_help_id"].name = "help_id"
master_catalogue.remove_column('brown_help_id')
    
for col in only_in_brown:
    master_catalogue["brown_{}".format(col)].name = col
    
for col in only_in_help:
    master_catalogue["help_{}".format(col)].name = col

brown_stellarity_origin is not a column of floats
brown_field is not a column of floats


In [10]:
for col in master_catalogue.colnames:
    if "m_" in col or "merr_" in col or "f_" in col or "ferr_" in col or "stellarity" in col:
        master_catalogue[col].fill_value = np.nan
    elif "flag" in col:
        master_catalogue[col].fill_value = 0
    elif "id" in col:
        master_catalogue[col].fill_value = -1
        
master_catalogue = master_catalogue.filled()

## 3  - Crop to HELP area

The Bootes Brown catalogue slightly goes outside the HELP Bootes field.

In [11]:
help_bootes_moc = MOC(filename="../../dmu2/dmu2_field_coverages/Bootes_MOC.fits")
in_HELP_mask = inMoc(
    master_catalogue['ra'], master_catalogue['dec'],
    help_bootes_moc 
)

master_catalogue = master_catalogue[in_HELP_mask]

## 4 - Check and save

In [12]:
master_catalogue[:10].show_in_notebook()

idx,help_id,f_ap_lbc_u,ferr_ap_lbc_u,m_ap_lbc_u,merr_ap_lbc_u,f_lbc_u,ferr_lbc_u,m_lbc_u,merr_lbc_u,f_ap_suprime_z,ferr_ap_suprime_z,m_ap_suprime_z,merr_ap_suprime_z,f_suprime_z,ferr_suprime_z,m_suprime_z,merr_suprime_z,f_ap_lbc_y,ferr_ap_lbc_y,m_ap_lbc_y,merr_ap_lbc_y,f_lbc_y,ferr_lbc_y,m_lbc_y,merr_lbc_y,field,ra,dec,hp_idx,f_ap_gpc1_g,ferr_ap_gpc1_g,m_ap_gpc1_g,merr_ap_gpc1_g,f_gpc1_g,ferr_gpc1_g,m_gpc1_g,merr_gpc1_g,f_ap_gpc1_r,ferr_ap_gpc1_r,m_ap_gpc1_r,merr_ap_gpc1_r,f_gpc1_r,ferr_gpc1_r,m_gpc1_r,merr_gpc1_r,f_ap_gpc1_i,ferr_ap_gpc1_i,m_ap_gpc1_i,merr_ap_gpc1_i,f_gpc1_i,ferr_gpc1_i,m_gpc1_i,merr_gpc1_i,f_ap_gpc1_z,ferr_ap_gpc1_z,m_ap_gpc1_z,merr_ap_gpc1_z,f_gpc1_z,ferr_gpc1_z,m_gpc1_z,merr_gpc1_z,f_ap_gpc1_y,ferr_ap_gpc1_y,m_ap_gpc1_y,merr_ap_gpc1_y,f_gpc1_y,ferr_gpc1_y,m_gpc1_y,merr_gpc1_y,f_ap_90prime_g,ferr_ap_90prime_g,m_ap_90prime_g,merr_ap_90prime_g,f_90prime_g,ferr_90prime_g,m_90prime_g,merr_90prime_g,f_ap_90prime_r,ferr_ap_90prime_r,m_ap_90prime_r,merr_ap_90prime_r,f_90prime_r,ferr_90prime_r,m_90prime_r,merr_90prime_r,f_ap_mosaic_z,ferr_ap_mosaic_z,m_ap_mosaic_z,merr_ap_mosaic_z,f_mosaic_z,ferr_mosaic_z,m_mosaic_z,merr_mosaic_z,f_ap_newfirm_j,ferr_ap_newfirm_j,m_ap_newfirm_j,merr_ap_newfirm_j,f_newfirm_j,ferr_newfirm_j,m_newfirm_j,merr_newfirm_j,f_ap_newfirm_h,ferr_ap_newfirm_h,m_ap_newfirm_h,merr_ap_newfirm_h,f_newfirm_h,ferr_newfirm_h,m_newfirm_h,merr_newfirm_h,f_ap_newfirm_k,ferr_ap_newfirm_k,m_ap_newfirm_k,merr_ap_newfirm_k,f_newfirm_k,ferr_newfirm_k,m_newfirm_k,merr_newfirm_k,f_ap_mosaic_r,ferr_ap_mosaic_r,m_ap_mosaic_r,merr_ap_mosaic_r,f_mosaic_r,ferr_mosaic_r,m_mosaic_r,merr_mosaic_r,f_ap_mosaic_i,ferr_ap_mosaic_i,m_ap_mosaic_i,merr_ap_mosaic_i,f_mosaic_i,ferr_mosaic_i,m_mosaic_i,merr_mosaic_i,f_ap_mosaic_b,ferr_ap_mosaic_b,m_ap_mosaic_b,merr_ap_mosaic_b,f_mosaic_b,ferr_mosaic_b,m_mosaic_b,merr_mosaic_b,f_ap_tifkam_ks,ferr_ap_tifkam_ks,m_ap_tifkam_ks,merr_ap_tifkam_ks,f_tifkam_ks,ferr_tifkam_ks,m_tifkam_ks,merr_tifkam_ks,f_ap_ukidss_j,ferr_ap_ukidss_j,m_ap_ukidss_j,merr_ap_ukidss_j,f_ukidss_j,ferr_ukidss_j,m_ukidss_j,merr_ukidss_j,f_ap_90prime_z,ferr_ap_90prime_z,m_ap_90prime_z,merr_ap_90prime_z,f_90prime_z,ferr_90prime_z,m_90prime_z,merr_90prime_z,f_ap_irac_i1,ferr_ap_irac_i1,m_ap_irac_i1,merr_ap_irac_i1,f_irac_i1,ferr_irac_i1,m_irac_i1,merr_irac_i1,f_ap_irac_i2,ferr_ap_irac_i2,m_ap_irac_i2,merr_ap_irac_i2,f_irac_i2,ferr_irac_i2,m_irac_i2,merr_irac_i2,f_ap_irac_i3,ferr_ap_irac_i3,m_ap_irac_i3,merr_ap_irac_i3,f_irac_i3,ferr_irac_i3,m_irac_i3,merr_irac_i3,f_ap_irac_i4,ferr_ap_irac_i4,m_ap_irac_i4,merr_ap_irac_i4,f_irac_i4,ferr_irac_i4,m_irac_i4,merr_irac_i4,stellarity,flag_cleaned,flag_merged,flag_gaia,flag_optnir_obs,flag_optnir_det,ebv,zspec_association_flag,zspec_qual,zspec,stellarity_origin
,,uJy,uJy,,mag,,,,,uJy,uJy,,mag,,,,,uJy,uJy,,mag,,,,,,deg,deg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,uJy,uJy,,,,,,,uJy,uJy,,,,,,,uJy,uJy,,,,,mag,mag,,,mag,mag,,,mag,mag,,,mag,mag,,,mag,mag,,,mag,mag,,,mag,mag,,,mag,mag,,,mag,mag,,,mag,mag,,,mag,mag,,,mag,mag,,,mag,mag,,,mag,mag,,,,,,,,,,,mag,mag,,,mag,mag,uJy,uJy,,,uJy,uJy,,,uJy,uJy,,,uJy,uJy,,,uJy,uJy,,,uJy,uJy,,,uJy,uJy,,,uJy,uJy,,,,,,,,,,,,,
0,HELP_J142948.065+344534.306,0.041397836059331894,0.06664155423641205,25.79438591003418,nan,0.041397836059331894,0.06664155423641205,27.357555899133324,1.7478002490788347,0.21058863401412964,0.11722900718450546,24.77898597717285,nan,0.21058863401412964,0.11722900718450546,25.59141268114498,0.6044000330023912,0.39251798391342163,0.6034532189369202,23.390090942382812,nan,0.39251798391342163,0.6034532189369202,24.915351101211222,1.669199971795852,Bootes,217.45027035616198,34.759529327634,172259325,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.38965428,1.0243291854858398,22.932276,nan,0.38965428,1.0243291854858398,2

In [13]:
columns = ["help_id", "field", "ra", "dec", "hp_idx"]

bands = [column[5:] for column in master_catalogue.colnames if 'f_ap' in column]
for band in bands:
    columns += ["f_ap_{}".format(band), "ferr_ap_{}".format(band),
                "m_ap_{}".format(band), "merr_ap_{}".format(band),
                "f_{}".format(band), "ferr_{}".format(band),
                "m_{}".format(band), "merr_{}".format(band),
                #"flag_{}".format(band)
               ]    
    
columns += ["stellarity",  "flag_cleaned", "flag_merged", "flag_gaia", "flag_optnir_obs", 
            "flag_optnir_det", "ebv", 'zspec_association_flag', 'zspec_qual', 'zspec', "stellarity_origin"] 

In [14]:
master_catalogue = master_catalogue.filled()
print(columns)
master_catalogue[columns].write("{}/master_catalogue_bootes{}.fits".format(OUT_DIR, SUFFIX), overwrite=True)

['help_id', 'field', 'ra', 'dec', 'hp_idx', 'f_ap_lbc_u', 'ferr_ap_lbc_u', 'm_ap_lbc_u', 'merr_ap_lbc_u', 'f_lbc_u', 'ferr_lbc_u', 'm_lbc_u', 'merr_lbc_u', 'f_ap_suprime_z', 'ferr_ap_suprime_z', 'm_ap_suprime_z', 'merr_ap_suprime_z', 'f_suprime_z', 'ferr_suprime_z', 'm_suprime_z', 'merr_suprime_z', 'f_ap_lbc_y', 'ferr_ap_lbc_y', 'm_ap_lbc_y', 'merr_ap_lbc_y', 'f_lbc_y', 'ferr_lbc_y', 'm_lbc_y', 'merr_lbc_y', 'f_ap_gpc1_g', 'ferr_ap_gpc1_g', 'm_ap_gpc1_g', 'merr_ap_gpc1_g', 'f_gpc1_g', 'ferr_gpc1_g', 'm_gpc1_g', 'merr_gpc1_g', 'f_ap_gpc1_r', 'ferr_ap_gpc1_r', 'm_ap_gpc1_r', 'merr_ap_gpc1_r', 'f_gpc1_r', 'ferr_gpc1_r', 'm_gpc1_r', 'merr_gpc1_r', 'f_ap_gpc1_i', 'ferr_ap_gpc1_i', 'm_ap_gpc1_i', 'merr_ap_gpc1_i', 'f_gpc1_i', 'ferr_gpc1_i', 'm_gpc1_i', 'merr_gpc1_i', 'f_ap_gpc1_z', 'ferr_ap_gpc1_z', 'm_ap_gpc1_z', 'merr_ap_gpc1_z', 'f_gpc1_z', 'ferr_gpc1_z', 'm_gpc1_z', 'merr_gpc1_z', 'f_ap_gpc1_y', 'ferr_ap_gpc1_y', 'm_ap_gpc1_y', 'merr_ap_gpc1_y', 'f_gpc1_y', 'ferr_gpc1_y', 'm_gpc1_y', 'me

## 5 - Make final cross ID catalogue

In [15]:
brown_cross_id = Table.read("{}/{}".format(OUT_DIR,brown_cross_id_name))
brown_cross_id['help_id'].name = 'brown_help_id'
brown_cross_id['full_help_id'].name = 'help_help_id'


help_cross_id = Table.read("{}/{}".format(OUT_DIR,help_cross_id_name))
help_cross_id.remove_columns(['specz_id', 'sdss_id'])
help_cross_id['help_id'].name = 'help_help_id'

final_cross_id = join(brown_cross_id, help_cross_id, join_type='left')





for col in final_cross_id.colnames:
    final_cross_id[col].fill_value = -1
final_cross_id = final_cross_id.filled()

final_cross_id['specz_id'][final_cross_id['specz_id'] == ''] = '-1'

final_cross_id.add_column(Column(data=final_cross_id['help_help_id'], name = 'help_id'))
have_brown_id = (final_cross_id['brown_help_id'] != '-1')
final_cross_id['help_id'][have_brown_id] = final_cross_id['brown_help_id'][have_brown_id]

In [16]:
final_cross_id.colnames

['brown_i_id',
 'brown_irac_id',
 'help_help_id',
 'brown_help_id',
 'specz_id',
 'sdss_id',
 'ps1_id',
 'legacy_id',
 'ibis_id',
 'ndwfs_id',
 'uhs_id',
 'zbootes_id',
 'datafusion_intid',
 'help_id']

In [17]:
columns = ['help_id',
           'brown_help_id',
           'help_help_id',
           'brown_i_id',
           'brown_irac_id',
           'specz_id',
           'sdss_id',
           'ps1_id',
           'legacy_id',
           'ibis_id',
           'ndwfs_id',
           'uhs_id',
           'zbootes_id',
           'datafusion_intid']
print(set(columns) - set(final_cross_id.colnames))
final_cross_id = final_cross_id[columns]

set()


In [18]:
final_cross_id[:10].show_in_notebook()

idx,help_id,brown_help_id,help_help_id,brown_i_id,brown_irac_id,specz_id,sdss_id,ps1_id,legacy_id,ibis_id,ndwfs_id,uhs_id,zbootes_id,datafusion_intid
0,HELP_J142500.285+331035.838,HELP_J142500.285+331035.838,-1,394263,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,HELP_J143318.051+332158.118,HELP_J143318.051+332158.118,-1,473154,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,HELP_J142939.235+354635.105,HELP_J142939.235+354635.105,-1,2019839,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,HELP_J142446.142+331756.854,HELP_J142446.142+331756.854,-1,445407,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,HELP_J142639.609+354724.470,HELP_J142639.609+354724.470,-1,2027735,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,HELP_J142446.456+324136.432,HELP_J142446.456+324136.432,-1,161462,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
6,HELP_J142440.580+354503.991,HELP_J142440.580+354503.991,-1,2005436,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,HELP_J142742.433+325234.332,HELP_J142742.433+325234.332,-1,242716,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
8,HELP_J142510.784+344929.132,HELP_J142510.784+344929.132,-1,1446199,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
9,HELP_J143221.937+354511.185,HELP_J143221.937+354511.185,-1,2006597,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [19]:

final_cross_id.write("master_list_cross_ident_bootes{}.fits".format(SUFFIX), overwrite=True)